In [78]:
import os
import sys
""" module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
sys.path.append(module_path) """
import warnings
import torch
import torch as tc
from torch.func import jacfwd, vmap, grad,jacrev

from Model import Model
from matrixM import MatrixM
from vecF import VecF
from PDE import KdV


""" def f(x, y):
  return x + y ** 2
x, y = torch.randn(5), torch.randn(5)
jacobian = jacfwd(lambda x,y: x+y**2, argnums=1)(x, y)
expected = torch.diag(2 * y)
assert torch.allclose(jacobian, expected)
jacobian
 """


' def f(x, y):\n  return x + y ** 2\nx, y = torch.randn(5), torch.randn(5)\njacobian = jacfwd(lambda x,y: x+y**2, argnums=1)(x, y)\nexpected = torch.diag(2 * y)\nassert torch.allclose(jacobian, expected)\njacobian\n '

In [79]:

class Model(tc.nn.Module):
    
    def __init__(self, d, m, theta=None, kind='gaussian'):
        
        super().__init__()
        self.m = m       # model parameter 
        self.d = d       # dimension of b (same as x)
        
        # size of theta according to the the model
        if kind == 'gaussian':
            self.s = 2*m+d*m 
        elif kind== 'nn':
            self.L = 0.5
            self.l = 3
            self.s = m + m*d * self.l + d * self.l 
        else:
            self.s = 2*m+d*m
            warnings.warn("The type of network has not The theta parameter has not been initialized") 
        
        # Dictionary of models 
        methods = {
            'gaussian'        : self.gaussian_x,   # Eq. (16)
        }

        # Warning : une initialisation à ones fait échouer la minimisation pour theta0 !
        self.Theta = tc.rand(self.s,requires_grad=True) #if theta is None else theta
        self.theta = tc.nn.Parameter(tc.rand(self.s) if theta is None else theta)
         #self.theta = tc.nn.Parameter(2.0*tc.ones(self.s) if theta is None else theta) 

        # batched derivatives , input are (d,)
        self.eval_x    = methods[kind]
        self.forward   = vmap(self.eval_x)
        self.dx        = vmap(grad(self.eval_x),             in_dims=(0,) )
        self.dxx       = vmap(grad(grad(self.eval_x)),       in_dims=(0,) )
        self.dxxx      = vmap(grad(grad(grad(self.eval_x))), in_dims=(0,) )

        # work in progress
        # self.dtheta = vmap(lambda x : autograd(self.eval_x(x), self.theta)[0])

        
    def splitter_gaussian(self):
        c = self.theta[:self.m]
        b = self.theta[self.m: self.m+self.m*self.d ].reshape(self.m,self.d)
        w = self.theta[self.m+self.m*self.d:]
        return (c,b,w)
    def splitter_gaussian1(self):
        c = self.Theta[:self.m]
        b = self.Theta[self.m: self.m+self.m*self.d ].reshape(self.m,self.d)
        w = self.Theta[self.m+self.m*self.d:]
        return (c,b,w)
    
    def splitter_nn(self):
        c = self.theta[:self.m]
        b=self.theta[self.m:self.m+self.d*self.l].reshape(self.l,self.d)
        w=self.theta[self.m+self.d*self.l:].reshape(self.l,self.m,self.d)
        return (c,b,w)
    
    def gaussian_x(self, x: tc.Tensor):
               
       # Input :  a SCALAR x of shape (0,)
       # Output : a SCALAR
        
        c, b, w = self.splitter_gaussian()
        return tc.sum(c*tc.exp(-w**2 *(x-b).T**2))  

    
    """
    
    def forward_t(self, theta, x):
        self.theta = theta
        c,b,w = self.splitter_gaussian()
        diff = x.unsqueeze(2)-b.t().unsqueeze(0) # tensor with shape (npoints,d,m)
        distance = torch.norm(diff, dim=1)**2 # tensor of dimension (npoints, m)
        exp_factor = torch.exp(-w.unsqueeze(0)**2*distance)
        return torch.sum(c.unsqueeze(0)*exp_factor, dim=1).unsqueeze(1) """
    def forward_t(self, theta, x):
        self.Theta = theta.clone()
        c,b,w = self.splitter_gaussian1()
        #vect=lambda x: (c*tc.exp(-w**2*(x-b)**2)).sum()
        #return (c*tc.exp(-w**2*(x-b)**2)).sum()
        return (c*tc.exp(-w**2*(x-b)**2)).sum()
    
    def grad_theta(self, theta, samples):
        self.Theta = theta.clone()
        jacobian=jacfwd(self.forward_t,argnums=0)(self.Theta, samples)
        return jacobian
        
    def out_prod(self,theta,x):
        #self.Theta=theta.clone()
        out_p=lambda theta,x_: tc.outer(self.grad_theta(theta,x_),self.grad_theta(theta,x_))
        M=vmap((out_p),in_dims=(None,0))(theta,x)
        return tc.mean(M,dim=0)
    
    def __call__(self, x):
        return self.forward(x)
        

In [80]:
import time
start = time.time()


d = 1
m = 2
s=2*m+d*m 
theta=tc.rand(s,requires_grad=True)
print(theta)
NN = Model(d,m,theta)
samples = torch.rand(10, d)
result_grad = NN.grad_theta(theta, samples[1])
result_out = NN.out_prod(theta, samples)
print(result_out)

tensor([0.3169, 0.8238, 0.5502, 0.3625, 0.5025, 0.7829], requires_grad=True)
tensor([[ 3.8204e+00,  3.7001e+00, -3.9899e-01, -2.7961e-02, -1.0932e-01,
         -4.1885e-01],
        [ 3.7001e+00,  3.5848e+00, -3.8494e-01, -2.5010e-02, -1.0463e-01,
         -4.0098e-01],
        [-3.9899e-01, -3.8494e-01,  1.2599e-01,  8.7895e-02,  1.2755e-02,
          4.8916e-02],
        [-2.7961e-02, -2.5010e-02,  8.7895e-02,  8.6085e-02,  2.6807e-03,
          1.0266e-02],
        [-1.0932e-01, -1.0463e-01,  1.2755e-02,  2.6807e-03,  4.2777e-03,
          1.6279e-02],
        [-4.1885e-01, -4.0098e-01,  4.8916e-02,  1.0266e-02,  1.6279e-02,
          6.1964e-02]], grad_fn=<MeanBackward1>)


In [81]:
M1=MatrixM(d,m,samples)
print(M1(theta))

tensor([0.3169, 0.8238, 0.5502, 0.3625, 0.5025, 0.7829], requires_grad=True)
tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])


In [82]:
pde = KdV(T = 4, xmin=-10, xmax=20, numberTimes=300)
t=1
VF=VecF(pde.d,NN.m,pde.f,samples)
print(VF(t,theta))

tensor([0., 0., 0., 0., 0., 0.], grad_fn=<MeanBackward1>)


In [83]:
print(theta)
MM=NN.out_prod(theta,samples)
#print(MM)
M_1=MatrixM(d,m,samples)
MM1=M_1(theta)
print(MM1)
#print(MM1)
g=tc.allclose(MM,MM1)
#print(g) 
pde = KdV(T = 4, xmin=-10, xmax=20, numberTimes=300)
#Fapprox = VecF(self.pde.d, self.model.m, self.pde.f, samples, kind = self.kind)
VF=VecF(pde.d,NN.m,pde.f,samples)
t=0.1
#NN(sample)
print(VF(t,theta))


tensor([0.3169, 0.8238, 0.5502, 0.3625, 0.5025, 0.7829], requires_grad=True)
tensor([0.3169, 0.8238, 0.5502, 0.3625, 0.5025, 0.7829], requires_grad=True)
tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
tensor([0., 0., 0., 0., 0., 0.], grad_fn=<MeanBackward1>)


In [84]:
""" tensorproduct = torch.zeros(samples.size()[0], NN.theta.size()[0], NN.theta.size()[0])
for iter in range(0, samples.size()[0]):
    tensorproduct[iter] = torch.outer(result_grad[iter], result_grad[iter]) 
    
torch.mean(tensorproduct, dim=0).shape """

' tensorproduct = torch.zeros(samples.size()[0], NN.theta.size()[0], NN.theta.size()[0])\nfor iter in range(0, samples.size()[0]):\n    tensorproduct[iter] = torch.outer(result_grad[iter], result_grad[iter]) \n    \ntorch.mean(tensorproduct, dim=0).shape '

In [85]:
""" def u(theta,x):
    return x**2*theta.dot(theta)
n=10
space_x=tc.linspace(0,4,100)
theta=tc.rand(n)
def grad_u(theta,x):
    return jacrev(u,argnums=0)(theta,x)
def out_prod(theta,vect_x):
    M=lambda vect,x: tc.outer(grad_u(vect,x),grad_u(vect,x))
    return vmap((M),in_dims=(None,0))(theta,vect_x)
    
M=tc.mean(out_prod(theta,space_x),dim=0)
print(M.shape) """

' def u(theta,x):\n    return x**2*theta.dot(theta)\nn=10\nspace_x=tc.linspace(0,4,100)\ntheta=tc.rand(n)\ndef grad_u(theta,x):\n    return jacrev(u,argnums=0)(theta,x)\ndef out_prod(theta,vect_x):\n    M=lambda vect,x: tc.outer(grad_u(vect,x),grad_u(vect,x))\n    return vmap((M),in_dims=(None,0))(theta,vect_x)\n    \nM=tc.mean(out_prod(theta,space_x),dim=0)\nprint(M.shape) '

In [86]:
vect_x=tc.rand(2,requires_grad=True)
vect_y=vect_x.clone()
print(vect_y.requires_grad)

True
